# Question 2 : Part 2
- Read all the json files in the folder called Data.
    - Read only the json files which contain the key called “restaurants” 
    - Each (or most of the json files) contain a key called “open” which contains the details of the operation (timings) of the restaurants. For each json file, read the timings of the restaurants. 
    - Data of the operation timings of the restaurants is present for each day of the week. I want you to extract each of this data and write it in an excel sheet. 
    - Bonus: Split “Start Time” column into two different columns having the hour and minute in each of them
    - Split “End Time” column into two different columns having the hour and minute in each of them


In [1]:
# importing all the packages needed 
import json
import glob
import os
import shutil
import re
import csv

In [2]:
path = os.getcwd() # getting the path of current working directory to create the file
configfiles = glob.glob(os.path.join(os.getcwd() + '\Data' + '\*.json')) # getting the path of the json files to be read

In [3]:
# function filters the json files by returning only the restuarants files 
def filteringRestuarantsFiles(configfiles):
    files = []
    for file1 in configfiles: # iterating through the files
         with open(file1) as json_file: # opening the file
            json_data = json.load(json_file) # reading the json data
            if json_data['term'] == 'restaurants': # checking whether its a restuarant file
                if 'hours' in json_data.keys(): # checking if the file has hours tag
                    hours = json_data['hours'] # taking the value of hours
                    for each in hours: # iterating through the value which is a list
                        if 'open' in each.keys(): # checking if open tag is present if it has
                            files.append(file1) # append the file to the list
    return files # returns the restuarant files

In [4]:
# function returns the list of Name, County, City, Timings per day for each restuarants
def dataToWriteInCsv():
    restuarants = []
    filteredFiles = filteringRestuarantsFiles(configfiles) # calling the function to get the restuarant files
    for file in filteredFiles: # iterating through the files
        with open(file) as json_file: # opening the file
            json_data = json.load(json_file) # reading the json data
            name = json_data['name'] # get the name of restuarant
            name = name.replace(',','') # replace with space if the name of contains ',' as we are writing to a csv file
            location = json_data['location'] # get the location of restuarant
            hours = json_data['hours'] # get the hours of restuarant
            city = location['city'] # get the city of restuarant
            country = location['country'] # get the country of restuarant
            for days in hours: # iterating through hours
                openWeeks = days['open'] # get the open of hours
                for each in  openWeeks: # iterating through the open tag
                    day = each['day'] # get the day restuarant is open
                    startTime = each['start'] # get the start time of restuarant
                    endTime = each['end'] # get the end time of restuarant
                    startHrAndMin = re.findall('..',startTime) # spliting the hour and min for start time
                    endHrAndMin = re.findall('..',endTime) # spliting the hour and min for end time
                    openHours = [] 
                    openHours.append(name) # appending the name
                    openHours.append(city) # appending the city
                    openHours.append(country) # appending the country
                    openHours.append(day) # appending the day
                    openHours.extend(startHrAndMin) # extending the hr and min of start time
                    openHours.extend(endHrAndMin) # extending  the hr and min of end time
                    restuarants.append(openHours) # appending the openhours to new list
    return restuarants # returns the list of all timings of restuarants

In [5]:
# function writing the csv file with restuarants details
def csvCreation(filename, restuarants):
    with open(filename, 'a',encoding='utf_8', errors='ignore') as outcsv: # opening a csv file as english file
        #configure writer to write standard csv file
        writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        # writing the headers for the file
        writer.writerow(['NAME OF RESTAURANT', 'CITY', 'COUNTRY CODE', 'DAY OF WEEK', 'START TIME HOUR','START TIME MINUTE', 'END TIME HOUR', 'END TIME MINUTE'])
        for eachRes in restuarants:
            # writing the data row by row
            writer.writerow([eachRes[0],eachRes[1],eachRes[2],eachRes[3],eachRes[4],eachRes[5],eachRes[6],eachRes[7]])

In [6]:
# creating the csv file at specified path
def output(csvPath):
    if os.path.exists(csvPath):
        os.remove(csvPath) # removing the file if already present
        restuarants = dataToWriteInCsv() # creating the restuarants data
        csvCreation(csvPath, restuarants) # creating the csv file
    else:
        restuarants = dataToWriteInCsv()  # creating the restuarants data
        csvCreation(csvPath, restuarants) # creating the csv file

In [7]:
csvPath = os.path.join(path + '\\' + 'Restuarants.csv') # path is taken

In [8]:
output(csvPath) # calling the function to create csv file